In [1]:
# importação das bibliotecas
import os
import numpy as np

In [2]:
# configuração dos hiperparametros
class Hp():
    def __init__(self):
        self.nb_steps = 1000 # numero de episódios
        self.episode_lenght = 1000 # máxima duração dos episódios
        self.learning_rate = 0.02 # taxa de aprendizado
        self.nb_directions = 16 # numero de direções, matrizes de perturbações contruidas
        self.nb_best_directions = 16 # mellhores recompensas
        assert self.nb_best_directions <= self.nb_directions # garantir que o best directions é menor que o directions
        self.noise = 0.03 # ruido, filtro de variancia nos dados
        self.seed = 1 # manter os resultados
        self.env_name = ''

In [3]:
# normalização dos estados
class Normalizer():
    def __init__(self, nb_inputs):
        self.n = np.zeros(nb_inputs) # numero de entradas, inicializando com valores 0
        self.mean = np.zeros(nb_inputs) 
        self.mean_diff = np.zeros(nb_inputs)
        self.var = np.zeros(nb_inputs)
    
    def observ(self, x): # x será um vetor
        self.n += 1.
        last_mean = self.mean.copy()
        self.mean += (x - self.mean) / self.n
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean / self.n).clip(min = 1e-2) # não pode ser menor que 0.001

    def normailize(self, inputs):
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        return (inputs - obs_mean) / obs_std    

In [ ]:
# contrução da Inteligência
class Policy():
    def __init__(self, input_size, output_size):
        self.theta = np.zeros((output_size, input_size)) # matriz de pesos
    
    def evaluate(self, input, delta = None, direction = None):
        if direction is None:
            return self.theta.dot(input)
        
        elif direction == 'positive':
            return (self.theta + hp.noise * delta).dot(input)

        else:
            return (self.theta - hp.noise * delta).dot(input)

